Run the database_setup script first. Change the mysql password in both files if necessary.

Then run initial_upload.ipynb to load the original csv into the database.

In [3]:
# import libraries
import mysql.connector
import pandas as pd

from mysql.connector import errorcode

# set DB_NAME here
DB_NAME = 'timperio'

# define function to create database if required
def create_database(cursor):
    try:
        cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
# Connect to database, creating the database if it does not exist
connection = mysql.connector.connect(host="localhost", user="root", password="") # change the password to what is set in your local environment
cursor = connection.cursor()

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        connection.database = DB_NAME
    else:
        print(err)
        exit(1)

In [ ]:
# Define tables as a dictionary of values
TABLES = {}

TABLES['users'] = (
    "CREATE TABLE `users` ("
    "  `user_id` INT NOT NULL AUTO_INCREMENT,"
    "  `username` VARCHAR(32) NOT NULL,"
    "  `password` VARCHAR(32) NOT NULL,"
    "  `role` VARCHAR(32) NOT NULL," # TODO: Determine the roles enum and set it here
    "  PRIMARY KEY (`user_id`)"
    ") ENGINE=InnoDB")

TABLES['products'] = (
    "CREATE TABLE `products` ("
    "  `product_id` INT NOT NULL AUTO_INCREMENT,"
    "  `product_name` VARCHAR(100) NOT NULL,"
    "  `product_description` VARCHAR(100) NOT NULL DEFAULT 'No Description',"
    "  `product_variant` INT NOT NULL,"
    "  `product_price` DECIMAL(8,2) NOT NULL,"
    "  PRIMARY KEY (`product_id`), UNIQUE KEY `product_name_variant` (`product_name`, `product_variant`)"
    ") ENGINE=InnoDB")

TABLES['promotions'] = (
    "CREATE TABLE `promotions` ("
    "  `promotion_id` INT AUTO_INCREMENT,"
    "  `promotion_name` VARCHAR(100) NOT NULL,"
    "  `promotion_description` VARCHAR(100) NOT NULL,"
    "  `promotion_type` VARCHAR(100) NOT NULL," # TODO: Use enum for promotion_type
    "  `valid_until` DATE NOT NULL,"
    "  `main_product_id` INT NOT NULL,"
    "  `discount_rate` DECIMAL(10,2),"
    "  `free_quantity` INT,"
    "  `promo_product_id` INT,"
    "  PRIMARY KEY (`promotion_id`), UNIQUE KEY `promotion_name` (`promotion_name`),"
    "  CONSTRAINT FK_PromotionMainProduct FOREIGN KEY (`main_product_id`) REFERENCES `products` (`product_id`) ON DELETE CASCADE,"
    "  CONSTRAINT FK_PromotionPromoProduct FOREIGN KEY (`promo_product_id`) REFERENCES `products` (`product_id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['customers'] = (
    "CREATE TABLE `customers` ("
    "  `customer_id` INT NOT NULL AUTO_INCREMENT,"
    "  `zipcode` INT NOT NULL DEFAULT '0',"
    "  `last_purchase_date` DATE,"
    "  `accept_newsletter` TINYINT(1) NOT NULL DEFAULT 0,"
    "  `email` VARCHAR(100),"
    "  PRIMARY KEY (`customer_id`)"
    ") ENGINE=InnoDB")

TABLES['newsletter_templates'] = (
    "CREATE TABLE `newsletter_templates` ("
    "  `template_id` INT NOT NULL AUTO_INCREMENT,"
    "  `template_name` VARCHAR(100) NOT NULL,"
    "  `template_description` VARCHAR(100) NOT NULL,"
    "  `template_content` VARCHAR(100) NOT NULL,"
    "  `created_at` DATE NOT NULL,"
    "  PRIMARY KEY (`template_id`)"
    ") ENGINE=InnoDB")

TABLES['newsletters'] = (
    "CREATE TABLE `newsletters` ("
    "  `newsletter_id` INT NOT NULL AUTO_INCREMENT,"
    "  `newsletter_name` VARCHAR(100) NOT NULL,"
    "  `newsletter_description` VARCHAR(100) NOT NULL,"
    "  `newsletter_content` VARCHAR(100) NOT NULL,"
    "  `created_at` DATE NOT NULL,"
    "  `template_id` INT NOT NULL,"
    "  PRIMARY KEY (`newsletter_id`),"
    "  CONSTRAINT FK_NewsletterTemplate FOREIGN KEY (`template_id`) REFERENCES `newsletter_templates` (`template_id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['customers_newsletters'] = (
    "CREATE TABLE `customers_newsletters` ("
    "  `customer_newsletter_id` INT NOT NULL AUTO_INCREMENT,"
    "  `customer_id` INT NOT NULL,"
    "  `newsletter_id` INT NOT NULL,"
    "  `send_success` BOOLEAN NOT NULL,"
    "  `send_date` DATE NOT NULL,"
    "  PRIMARY KEY (`customer_newsletter_id`),"
    "  CONSTRAINT FK_CustomerNewsletter_1 FOREIGN KEY (`customer_id`) REFERENCES `customers` (`customer_id`) ON DELETE CASCADE,"
    "  CONSTRAINT FK_CustomerNewsletter_2 FOREIGN KEY (`newsletter_id`) REFERENCES `newsletters` (`newsletter_id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['sales'] = (
    "CREATE TABLE `sales` ("
    "  `sale_id` INT NOT NULL AUTO_INCREMENT,"
    "  `sale_date` DATE NOT NULL,"
    "  `sale_type` VARCHAR(100) NOT NULL," # TODO: Use enum for sale_type
    "  `digital` VARCHAR(100) NOT NULL,"
    "  `shipping_method` VARCHAR(100) NOT NULL," # TODO: Use enum for shipping_method
    "  `product_id` INT NOT NULL,"
    "  `quantity` INT NOT NULL,"
    "  `original_price` DECIMAL(8,2) NOT NULL,"
    "  `customer_id` INT NOT NULL,"
    "  `promotion_id` INT,"
    "  `discounted_price` DECIMAL(8,2),"
    "  PRIMARY KEY (`sale_id`),"
    "  CONSTRAINT FK_SalesProduct FOREIGN KEY (`product_id`) REFERENCES `products` (`product_id`) ON DELETE CASCADE,"
    "  CONSTRAINT `FK_SalesCustomer` FOREIGN KEY (`customer_id`) REFERENCES `customers` (`customer_id`) ON DELETE CASCADE,"
    "  CONSTRAINT FK_SalesPromotion FOREIGN KEY (`promotion_id`) REFERENCES `promotions` (`promotion_id`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

# Create a table to track sequence of IDs
TABLES['sequence'] = (
    "CREATE TABLE `sequence` ("
    "  `table_name` VARCHAR(100) NOT NULL,"
    "  `next_id` INT NOT NULL,"
    "  PRIMARY KEY (`table_name`)"
    ") ENGINE=InnoDB")

In [7]:
# Create the tables in the order stated in the dictionary
# If the table already exists, skip
# If not, create the table

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("Table already exists.")
        else:
            print(err.msg)
    else:
        print("Table {} created".format(table_name))
        
print("Database Creation Completed")


Creating table users: Table users created
Creating table products: Table products created
Creating table promotions: Table promotions created
Creating table customers: Table customers created
Creating table newsletter_templates: Table newsletter_templates created
Creating table newsletters: Table newsletters created
Creating table customers_newsletters: Table customers_newsletters created
Creating table sales: Table sales created
Creating table sequence: Table sequence created
Database Creation Completed


In [8]:
# close cursor and connection
cursor.close()
connection.close()